In [1]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import os
import numpy as np
from hyperparams import hypers
from models.yolo import Yolov2Tiny
from models.loss import YoloLossMultiBoxes
from train import train_fn
from dataprocess.dataset import *
from dataprocess.dataprocess import *



In [2]:

if hypers.DATASET_PATH == "LOCAL":
    drive_dir=os.path.join(os.getcwd(), "archive","data")
elif hypers.DATASET_PATH == "DRIVE":
    drive_dir=os.path.join(os.getcwd(),"drive","MyDrive","archive","data")



#Dataframe Creation
train_labels=os.path.join(drive_dir,"train_solution_bounding_boxes (1).csv")
train_imgs_dir=os.path.join(drive_dir,"training_images")
test_imgs_dir=os.path.join(drive_dir,"testing_images")
labels_dir=os.path.join(drive_dir,"label")
images_dir=os.path.join(drive_dir,"image")

imgs_list=list(sorted(os.listdir(train_imgs_dir)))
idxs=list(range(len(imgs_list)))
np.random.shuffle(idxs)

train_idx=idxs[:int(0.8*len(idxs))]
val_idx=idxs[int(0.8*len(idxs)):]


imgs_list=list(sorted(os.listdir(train_imgs_dir)))    



model = Yolov2Tiny(split_size=7, num_boxes=hypers.NUM_BOXES, num_classes=hypers.NUM_CLASSES).to(hypers.DEVICE)
optimizer = optim.Adam( model.parameters(), lr=hypers.LEARNING_RATE, weight_decay=hypers.WEIGHT_DECAY )
loss_fn = YoloLossMultiBoxes(S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES)



In [3]:
transform = Compose([transforms.Resize((448, 448)), transforms.ToTensor(),])

annotations=create_annotations(images_dir) # LIST LIKE: 0     vid_4_1000.jpg   vid_4_1000.txt
annotations_predict=create_annotations(test_imgs_dir) # LIST LIKE: 0     vid_4_1000.jpg   vid_4_1000.txt


dataset=VOCDataset(annotations, labels_dir, images_dir, S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES, transform=transform)
dataset_predict=VOCDatasetPredict(annotations_predict, test_imgs_dir, S=7, B=hypers.NUM_BOXES, C=hypers.NUM_CLASSES, transform=transform)


train_set, test_set = torch.utils.data.random_split(dataset, [300, 55])

train_loader = DataLoader(dataset=train_set, batch_size=hypers.BATCH_SIZE,
                            pin_memory=hypers.PIN_MEMORY, shuffle=True, drop_last=True)

test_loader = DataLoader(dataset=test_set, batch_size=hypers.BATCH_SIZE,
                        pin_memory=hypers.PIN_MEMORY, shuffle=True, drop_last=True)


predict_loader = DataLoader(dataset=dataset_predict, batch_size=hypers.BATCH_SIZE,
                        pin_memory=hypers.PIN_MEMORY, shuffle=True, drop_last=True)